# VGN Inspection

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path
import time

import numpy as np
import rospy
import torch

from vgn import *
from vgn.dataset import Dataset
from vgn.networks import load_network
from vgn_ros import vis

In [ ]:
dataset_dir = Path("../data/datasets/blocks")
model = Path("/home/michel/Developer/catkin_ws/src/vgn/vgn/data/models/200524-124604,net=conv,dataset=blocks,batch_size=32,lr=3e-04,/vgn_conv_100.pth")

finger_depth = 0.05
size = 6.0 * finger_depth
voxel_size = size / 40.0

In [ ]:
# initialize a ros node to communicate with rviz
rospy.init_node("dataset_inspection")

In [ ]:
# load the dataset and trained network weights
dataset = Dataset(dataset_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = load_network(model, device)

## Visualize detections

In [ ]:
i = np.random.choice(len(dataset))
tsdf, _, _ = dataset[i]

tic = time.time()
out = predict(tsdf, net, device)
out = process(out)
grasps, scores = select(out)
grasps = [from_voxel_coordinates(g, voxel_size) for g in grasps]
toc = time.time() - tic
print("Detected {} grasps in {} seconds".format(len(grasps), toc))


vis.clear()
vis.workspace(size)
vis.tsdf(tsdf.squeeze(), voxel_size)
vis.quality(out[0], voxel_size)
vis.grasps(grasps, scores, finger_depth)